# Umwandlung von .xml Daten in pd.DataFrame

## Importe

In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

## Import von .xml Daten aus "MDB_STAMMDATEN.XML"

#### Funktionen

In [2]:
# Parsen/Lesen der Daten in XML-Format
def read_xml(path):
   xml = ET.parse(path).getroot()
   return xml

# Extraktion von Daten aus dem XML-Code
def search_for_name(tree, attr):
   names = []
   for child in tree:
       if child.tag == attr:
           names += [child.text]
       else:
           names += search_for_name(child, attr)
   return names

#### Erster Test mit dem Attribut "Geschlecht" (--> neue Spalte)

In [3]:
if __name__=="__main__":
    MdBxml = read_xml("MDB_STAMMDATEN.XML")
    names = search_for_name(MdBxml,"GESCHLECHT")

# Überprüfung
print(names[:4])
print(len(names))

['männlich', 'männlich', 'weiblich', 'weiblich']
4073


#### Erste Ansätze für ein vorläufiges DataFrame mit den folgenden Spalten:

In [4]:
# Merkmale (--> "WP" steht für Wahlperioden)
liste_merk = ["ID", "NACHNAME", "VORNAME","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND", "WP"]

# Einmaliges zusammensetzten einer Liste von Listen mit den Ausprägungen der obigen Merkmale
column_list = []
for attr in liste_merk:
    if __name__=="__main__":
        MdBxml = read_xml("MDB_STAMMDATEN.XML")
        names = search_for_name(MdBxml, attr)
        column_list.append(names)

#### Nun wird die Kompatibilität der einzelnen Listen überprüft.

#### Es soll ein pd.DataFrame erstellt werden und dafür müssen die Spaltenlängen übereinstimmen.

In [5]:
print(len(column_list))

for i in column_list:
    print(len(i))

9
4073
4336
4336
4073
4073
4073
4073
4073
11600


#### Mit dieser verkürzten Liste lässt sich schon ein DataFrame erstellen ( Länge = 4073 Einträge, da die IDs definitiv eimalig sind). 

#### Etwas umständlich, aber wirksam wird ein erstes DataFrame zusammengestellt und ein erster Mangel behoben 

In [6]:
liste_merk_2 = ["ID","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND"]

df_first = pd.DataFrame(np.column_stack([column_list[0],column_list[3],column_list[4],column_list[5],column_list[6],column_list[7]]), columns = liste_merk_2)

# None values in "Geburtsland werden durch "Deutschland" ersetzt
df_first.loc[df_first["GEBURTSLAND"].isnull(), "GEBURTSLAND"] = "Deutschland"

df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland


#### Bei den Datentypen besteht noch Verbesserungsbedarf, dazu aber später mehr.

In [7]:
df_first.dtypes

ID              object
GESCHLECHT      object
PARTEI_KURZ     object
GEBURTSDATUM    object
BERUF           object
GEBURTSLAND     object
dtype: object

#### Mit Hilfe einer höheren Macht lässt sich der Parsing-Prozess weiterentwickeln.

#### Hier die Anwendung für die Vor- und Nachnamen:

In [8]:
#Funktion
def search_for_nameRoots(root):
    names = []
    for child in root:
        if child.tag == "NAMEN":
            names += [child]
        else:
            names += search_for_nameRoots(child)
    return names


nameRoots = search_for_nameRoots(MdBxml)

# Listen für Nach- und Vornamen
firstNameElements = []
for element in nameRoots:
    firstNameElements += [element[0].find('VORNAME').text]

lastNameElements = []
for element in nameRoots:
    lastNameElements += [element[0].find('NACHNAME').text]

# Spalten erstellen    
df_first["NACHNAME"] = lastNameElements
df_first["VORNAME"] = firstNameElements

#Die Integration ist geglückt!
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich


#### Nun muss der Parsing-Prozess nochmal umgestaltet werden, um zu einer spezifischen ID/einem spezifischen MdB die Nummern aller Wahlperioden zu finden. 
#### Langfrisitg soll auf diese Perioden umindiziert werden.

In [9]:
# Die folgende Funktion soll eine Liste mit Listen dieser nummerierten Wahlperioden zu jeden MdB produzieren


def search_for_wpRoots1(root1):

    # Nested-function: Entscheidend ist hier der Break beim nächsten "ID"-Tag, da diese für jedes MdB einzigartig sind.
    def search_for_wpRoots2(root2):
        wplist2 = []
        for child2 in root2:
            if child2.tag == "WP":
                wplist2 += [child2]
            elif child2.tag == "ID":
                break
            else:
                wplist2 += search_for_wpRoots2(child2)
        return wplist2
    # Ende nested-function
    
    wplist = []
    for child in root1:
        if child.tag == "WAHLPERIODEN":
            wplist += [search_for_wpRoots2(child)]
        else:
            wplist += search_for_wpRoots1(child)
    return wplist



# Anwenden
WProots = search_for_wpRoots1(MdBxml)
    
# Nun müssen noch die gewünschten Nummern aus dem Code extrahiert werden    

#Funktion (wie schon weiter oben bei anderen Merkmalen)
def extracting_WP(list1):
    WPelements = []
    for i in list1:
        sub_list = []
        for a in i:
            sub_list += [a.text]
        WPelements += [sub_list]
    return WPelements

# Anwenden
WPs = extracting_WP(WProots)

# Wie sieht's aus?
print(WPs[:4])
# Testen der neuen Liste auf längenmäßige Kompatibilität mit dem DF  
print(len(WPs))

[['5', '6', '7', '8', '9', '10', '11'], ['3', '4', '5', '6', '7'], ['2', '3', '4'], ['11', '12']]
4073


In [10]:
# Einfügen in das DataFrame
df_first["WAHLPERIODEN"] = WPs
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred,"[5, 6, 7, 8, 9, 10, 11]"
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst,"[3, 4, 5, 6, 7]"
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie,"[2, 3, 4]"
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else,"[11, 12]"
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich,"[15, 16, 12, 13, 14]"


#### Der gleiche Prozess wird nun auch für die Manadatsart gewählt.
#### In der Liste, die nun zu jedem MdB erstellt wird, kann man die Mandatsart zur jeweiligen Wahlperiode (Index wie in Wahlperioden) finden.

In [11]:
# Funktion (wie zuvor)


def search_for_MandatsArtRoots1(root):
    
    # Nested-function
    def search_for_MandatRoots2(root2):
        mandatlist2 = []
        for child2 in root2:
            if child2.tag == "MANDATSART":
                mandatlist2 += [child2]
            elif child2.tag == "ID":
                break
            else:
                mandatlist2 += search_for_MandatRoots2(child2)
        return mandatlist2
    # Ende nested-function
    
    mandatlist = []
    for child in root:
        if child.tag == "WAHLPERIODEN":
            mandatlist += [search_for_MandatRoots2(child)]
        else:
            mandatlist += search_for_MandatsArtRoots1(child)
    return mandatlist



MandatRoots = search_for_MandatsArtRoots1(MdBxml)


# Nun müssen noch die gewünschten Mandatsarten aus dem Code extrahiert werden    
def extracting_Mandate(list1):
    MandatElements = []
    for i in list1:
        sub_list = []
        for a in i:
            sub_list += [a.text]
        MandatElements += [sub_list]
    return MandatElements

Mandate = extracting_Mandate(MandatRoots)

# Wie sieht's hier aus?
print(Mandate[:3])
# Testen der neuen Liste auf längenmäßige Kompatibilität mit dem DF  
print(len(Mandate))

[['Direktwahl', 'Direktwahl', 'Direktwahl', 'Direktwahl', 'Direktwahl', 'Direktwahl', 'Direktwahl'], ['Landesliste', 'Landesliste', 'Landesliste', 'Landesliste', 'Landesliste'], ['Landesliste', 'Landesliste', 'Landesliste']]
4073


In [12]:
# Einfügen in das DataFrame
df_first["MANDATSARTEN"] = Mandate
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred,"[5, 6, 7, 8, 9, 10, 11]","[Direktwahl, Direktwahl, Direktwahl, Direktwah..."
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst,"[3, 4, 5, 6, 7]","[Landesliste, Landesliste, Landesliste, Landes..."
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie,"[2, 3, 4]","[Landesliste, Landesliste, Landesliste]"
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else,"[11, 12]","[Volkskammer, Landesliste]"
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich,"[15, 16, 12, 13, 14]","[Direktwahl, Direktwahl, Direktwahl, Direktwah..."


## Data Cleaning


Das Dataframe muss nun gecleant werden, um gut für die Visualisierung genutzt werden zu können. 

Hierzu muss:
- Parteien sinnvoll zusammengefasst werden
- geguckt werden, ob ein drittes Geschlecht vorliegt und mit Boolean Werten ersetzt werden
- ID auf Dopplungen überprüft werden
- Geburtsdatum mit Alter ersetzt werden
- Geburtsland mit Boolean Werten ersetzt werden
- Dopplungen von Abgeordneten überprüft werden
- Mandatsarten mit Boolean Werten ersetzt werden (0=Direktwahl, 1=Landesliste, Volkskammer wird zu Landesliste gezählt)
- Einzelne Columns für die WP erstellt werden, um pivotisieren zu können
- Die Anzahl der Abgeordneten in jeder Wahlperiode mit historischen Daten abgeglichen werden

In [13]:
# Gucken, welche Parteien vorhanden sind
df_first['PARTEI_KURZ'].unique()

array(['CDU', 'FDP', 'SPD', 'KPD', 'DP', 'CSU', 'DZP', 'FU', 'GRÜNE',
       'BP', 'BÜNDNIS 90/DIE GRÜNEN', 'GB/ BHE', 'DIE GRÜNEN/BÜNDNIS 90',
       'PDS', 'PDS/LL', 'DIE LINKE.', 'SRP', 'Plos', None, 'WAV', 'DRP',
       'CSUS', 'DSU', 'DPB', 'CVP', 'DPS', 'AfD', 'Blaue'], dtype=object)

Glossar:

* KPD = Kommunistische Partei Deutschlands
* DP = Deutsche Partei
* DZP = Deutsche Zentrumspartei
* FU = Föderalistische Union (Fraktion aus BP und DZP)
* BP = Bayernpartei
* GB/ BHE = Gesamtdeutscher Block/Bund der Heimatvertriebenen und Entrechteten
* PDS = Partei des Demokratischen Sozialismus 
* PDS/LL = Partei des Demokratischen Sozialismus - Landesliste
* SRP = Sozialistische Reichspartei
* Plos = Parteilos
* WAV = Wirtschaftliche Aufbau-Vereinigung
* DRP = Deutsche Reichspartei
* CSUS = CSU-Saar
* DSU = Deutsche Soziale Union
* DPB = Deutsche Partei Bayern
* CVP = Christdemokratische Volkspartei des Saarlandes
* DPS = Demokratische Partei Saar (Saarpartei der FDP)
* Blaue = Blaue Partei

None Einträge müssen gefüllt oder gedropt werden

CDU, CSU, CSUS, CVP -> Union

Grüne, Bündnis 90/Die Grünen, Die Grünen/Bündnis 90 -> Bündnis 90/Die Grünen

PDS, PDS/LL, Die Linke -> Die Linke

FDP, DPS -> FDP

AfD, Blaue -> AfD

KPD, DP, DZP, FU, BP, GB/ BHE, SRP, WAV, DRP, DSU, DPB, Plos -> Andere

In [14]:
# Zusammenfassen von Parteien
df_first['PARTEI_KURZ'].replace(["CDU", "CSU", "CSUS", "CVP"],["Union", "Union", "Union", "Union"], inplace = True)
df_first['PARTEI_KURZ'].replace(["GRÜNE", "BÜNDNIS 90/DIE GRÜNEN", "DIE GRÜNEN/BÜNDNIS 90"],["Bündnis 90/Die Grünen", "Bündnis 90/Die Grünen", "Bündnis 90/Die Grünen"], inplace = True)
df_first['PARTEI_KURZ'].replace(["PDS", "PDS/LL", "DIE LINKE."],["Die Linke", "Die Linke", "Die Linke"], inplace = True)
df_first['PARTEI_KURZ'].replace(["FDP", "DPS"],["FDP", "FDP"], inplace = True)
df_first['PARTEI_KURZ'].replace(["AfD", "Blaue"],["AfD", "AfD"], inplace = True)
df_first['PARTEI_KURZ'].replace(["KPD", "DP", "DZP", "FU", "BP", "GB/ BHE", "SRP", "WAV", "DRP", "DSU", "DPB", "Plos"],["Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere", "Andere"], inplace = True)

In [15]:
# Füllen von None Werten mit Andere
df_first.PARTEI_KURZ.fillna(value="Andere", inplace=True)

In [16]:
# Das Zusammenfassen ist geglückt
df_first['PARTEI_KURZ'].unique()

array(['Union', 'FDP', 'SPD', 'Andere', 'Bündnis 90/Die Grünen',
       'Die Linke', 'AfD'], dtype=object)

In [17]:
# Überpüfung der Anzahl an Geschlechtern
df_first['GESCHLECHT'].unique()

array(['männlich', 'weiblich'], dtype=object)

In [18]:
# Überprüfung auf Dopplungen der ID
df_first[df_first.duplicated(subset="ID", keep=False)]

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN


In [19]:
# Überprüfung auf Dopplungen der Namen
df_first[df_first.duplicated(subset=["NACHNAME", "VORNAME", "GEBURTSDATUM"], keep=False)]

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN


In [20]:
# Überpüfung auf Korrektheit der Geburtsjahre
df_first['GEBURTSDATUM'].unique()

array(['1930', '1909', '1913', '1933', '1950', '1919', '1912', '1876',
       '1944', '1920', '1941', '1904', '1922', '1879', '1902', '1924',
       '1936', '1890', '1901', '1943', '1914', '1896', '1947', '1949',
       '1928', '1946', '1889', '1888', '1955', '1934', '1940', '1951',
       '1923', '1942', '1880', '1939', '1932', '1925', '1918', '1894',
       '1927', '1882', '1899', '1931', '1895', '1898', '1964', '1915',
       '1921', '1893', '1938', '1917', '1926', '1911', '1891', '1903',
       '1937', '1905', '1900', '1929', '1910', '1957', '1908', '1907',
       '1897', '1883', '1945', '1954', '1906', '1948', '1952', '1935',
       '1956', '1959', '1887', '1884', '1885', '1966', '1953', '1892',
       '1886', '1916', '1958', '1960', '1965', '1961', '1963', '1881',
       '1877', '1875', '1878', '1962', '1971', '1968', '1967', '1969',
       '1972', '1973', '1970', '1976', '1975', '1974', '1977', '1983',
       '1978', '1980', '1979', '1981', '1982', '1986', '1985', '1984',
      

In [21]:
# Überpüfung auf Korrektheit der Geburtsorte
df_first['GEBURTSLAND'].unique()

array(['Deutschland', 'Jugoslawien', 'Australien', 'Kenia', 'Spanien',
       'CSSR', 'Ukraine', 'Finnland', 'Dänemark', 'Österreich',
       'Lettland', 'Oberösterreich',
       'Tansania (ehem. Deutsch-Ost-Afrika)', 'Niederlande',
       'Niederösterreich', 'Estland', 'ehem. Deutsch-Ost-Afrika',
       'Tschechoslowakei', 'Slowakei', 'Ceylon', 'Rumänien', 'Perú',
       'Schweiz', 'Israel', 'Ungarn', 'Frankreich', 'Tonga-Insel, Südsee',
       '(damalige) CSR', 'Tansania', 'USA', 'CSFR', 'Polen',
       'Südwestafrika', 'Türkei', 'Italien', 'Kroatien', 'Iran',
       'Russland'], dtype=object)

In [22]:
# Um unser DF analysierbarer zu machen legen wir nun boolean-Spalten an.
# Alle Geburtsländer, die nicht Deutschland sind, werden der Einfachheit halber zusammengefasst und mit 0 kodiert
df_first["AUSLAND"] = df_first["GEBURTSLAND"] != "Deutschland"
df_first['AUSLAND'].replace([True, False], [1,0], inplace = True)

In [23]:
# Ähnliche Umgestaltung auch für die Spalte mt dem Geschlecht
df_first['GESCHLECHT'].replace(["männlich", "weiblich"],[1,0], inplace = True)

In [24]:
# Das Gleiche noch einmal für Mandatsart: Volkskammer wird hier zu Landesliste zusammengefasst
mandate = list(df_first['MANDATSARTEN'])
dictionarym = {"Direktwahl": 1, "Landesliste": 0, "Volkskammer": 0}

def replace_mandate(mandate_list, dictionary):
    mandate_list = [[dictionary.get(item, item) for item in lst] for lst in mandate_list]
    return mandate_list

mandate = replace_mandate(mandate, dictionarym)
df_first["MANDATSARTEN"] = mandate

## Pivotisierung, Feature Engineering und Export

#### Das Cleaning ist abgeschlossen, jetzt geht es an die Pivotisierung

In [25]:
dfnew = df_first.copy()
dfnew.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,AUSLAND
0,11000001,1,Union,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred,"[5, 6, 7, 8, 9, 10, 11]","[1, 1, 1, 1, 1, 1, 1]",0
1,11000002,1,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst,"[3, 4, 5, 6, 7]","[0, 0, 0, 0, 0]",0
2,11000003,0,Union,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie,"[2, 3, 4]","[0, 0, 0]",1
3,11000004,0,Union,1933,Ärztin,Deutschland,Ackermann,Else,"[11, 12]","[0, 0]",0
4,11000005,1,Union,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich,"[15, 16, 12, 13, 14]","[1, 1, 1, 1, 1]",0


#### Für die Pivotisierung wollen wir jetzt 19 Spalten für die verschiedenen Wahlperioden erstellen und diese mit Elementen aus [1,0] besetzen.


In [26]:
# Wie viele Wahlperioden gibt es?

unique_list = []
for e in dfnew["WAHLPERIODEN"]:
    for i in e:
        if int(i) in unique_list:
            None
        else:
            unique_list.append(int(i))
            
unique_list.sort()            
print(unique_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


#### Erstellen der 19 Spalten:

In [27]:
# Funktion

def WPcolumnchecker(column, range1):
    
    # Diese Funktion soll eine iteration über die Listen in der Spalte "Wahlperioden" durchführen
    def listchecker(WPNum, WPList):
        if WPNum in WPList:
            return 1
        else:
            return 0
    # Ende
    for n in range1:
        WPstr = "WP" + str(n)
        binary_list = []
        nstr= str(n)
        for index in range(0,4073):
            WPListe = column.loc[index]
            binary_list.append(listchecker(nstr, WPListe))
        dfnew[WPstr] = binary_list
        

        
# Anwendung
dfrange1 = unique_list
dfcolumn = dfnew["WAHLPERIODEN"]

WPcolumnchecker(dfcolumn, dfrange1)

#### Überprüfung für ein Beispiel
- Sind die 1 und 0 an den richtigen Stellen? --> z.B WP3 = 1 

In [28]:
# Überprüfung an einem Beispiel --> sind die 1 und 0 an den richtigen Stellen? --> z.B WP3 = 1 
print(dfnew.loc[1])

ID                            11000002
GESCHLECHT                           1
PARTEI_KURZ                        FDP
GEBURTSDATUM                      1909
BERUF           Rechtsanwalt und Notar
GEBURTSLAND                Deutschland
NACHNAME                     Achenbach
VORNAME                          Ernst
WAHLPERIODEN           [3, 4, 5, 6, 7]
MANDATSARTEN           [0, 0, 0, 0, 0]
AUSLAND                              0
WP1                                  0
WP2                                  0
WP3                                  1
WP4                                  1
WP5                                  1
WP6                                  1
WP7                                  1
WP8                                  0
WP9                                  0
WP10                                 0
WP11                                 0
WP12                                 0
WP13                                 0
WP14                                 0
WP15                     

#### Um ein DF nach den Wahlperioden zu indizieren, müssen zunächst die "Bundestage 1 bis 19" aus dem Mitgliederpool zusammen gestellt werden

In [29]:
# Wenn irgendwer Ahnung hat, wie man Namen von Variablen generieren kann, BITTE ÄNDERN!
df_WP1 = dfnew.loc[dfnew["WP1"] == 1]
df_WP2 = dfnew.loc[dfnew["WP2"] == 1]
df_WP3 = dfnew.loc[dfnew["WP3"] == 1]
df_WP4 = dfnew.loc[dfnew["WP4"] == 1]
df_WP5 = dfnew.loc[dfnew["WP5"] == 1]
df_WP6 = dfnew.loc[dfnew["WP6"] == 1]
df_WP7 = dfnew.loc[dfnew["WP7"] == 1]
df_WP8 = dfnew.loc[dfnew["WP8"] == 1]
df_WP9 = dfnew.loc[dfnew["WP9"] == 1]
df_WP10 = dfnew.loc[dfnew["WP10"] == 1]
df_WP11 = dfnew.loc[dfnew["WP11"] == 1]
df_WP12 = dfnew.loc[dfnew["WP12"] == 1]
df_WP13 = dfnew.loc[dfnew["WP13"] == 1]
df_WP14 = dfnew.loc[dfnew["WP14"] == 1]
df_WP15 = dfnew.loc[dfnew["WP15"] == 1]
df_WP16 = dfnew.loc[dfnew["WP16"] == 1]
df_WP17 = dfnew.loc[dfnew["WP17"] == 1]
df_WP18 = dfnew.loc[dfnew["WP18"] == 1]
df_WP19 = dfnew.loc[dfnew["WP19"] == 1]

# Beispiel
print("Es gab " + str(len(df_WP19)) + " Abgeordnete im 19.Bundestag")
df_WP19.head()

Es gab 711 Abgeordnete im 19.Bundestag


,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,...,WP10,WP11,WP12,WP13,WP14,WP15,WP16,WP17,WP18,WP19
195,11000198,1,Union,1952,"Landwirtschaftsmeister, Parl. Staatssekretär",Deutschland,Bleser,Peter,"[12, 13, 14, 15, 16, 17, 18, 19]","[1, 1, 1, 1, 1, 1, 1, 1]",...,0,0,1,1,1,1,1,1,1,1
360,11000365,1,Die Linke,1950,"Autor, Geschäftsführer",Deutschland,Dehm-Desoi,Jörg-Diether,"[12, 16, 17, 18, 19]","[0, 0, 0, 0, 0]",...,0,0,1,0,0,0,1,1,1,1
609,11000616,1,Union,1952,Parl. Staatssekretär,Deutschland,Fuchtel,Hans-Joachim,"[11, 12, 13, 14, 15, 16, 17, 18, 19]","[1, 1, 1, 1, 1, 1, 1, 1, 1]",...,0,1,1,1,1,1,1,1,1,1
749,11000756,1,Die Linke,1948,Rechtsanwalt,Deutschland,Gysi,Gregor,"[11, 12, 13, 14, 16, 17, 18, 19]","[0, 1, 1, 1, 1, 1, 1, 1]",...,0,1,1,1,1,0,1,1,1,1
1014,11001023,0,Die Linke,1951,"Dipl.-Soziologin, Journalistin",Deutschland,Jelpke,Ulla,"[12, 13, 14, 16, 17, 18, 19]","[0, 0, 0, 0, 0, 0, 0]",...,0,0,1,1,1,0,1,1,1,1


#### Als nächsten Schritt werden wir nun die 19 Bundestage(DFs) zusammenfügen und nach den Wahlperioden indizieren

In [30]:
# Vorbereitungen
unique_list_wp = []
for e in unique_list:
    unique_list_wp.append("WP" + str(e))

# Index
list_years = ["1949-1953;01", "1953–1957;02","1957–1961;03", "1961–1965;04", "1965–1969;05", "1969–1972;06", "1972–1976;07","1976-1980;08",
"1980–1983;09", "1983–1987;10","1987–1990;11","1990–1994;12","1994–1998;13","1998–2002;14","2002–2005;15","2005–2009;16","2009–2013;17",
              "2013–2017;18", "2017-;19"]              

dict_WPyears = dict(zip(unique_list_wp, list_years))

# Liste zum einfacheren Abruf
df_wp_list = [df_WP1, df_WP2, df_WP3, df_WP4, df_WP5, df_WP6, df_WP7,df_WP8,df_WP9, df_WP10,df_WP11,df_WP12,df_WP13,
df_WP14,df_WP15,df_WP16,df_WP17,df_WP18, df_WP19]

# Transformation / Pivotisierung der einzelnen DataFrame
for d in unique_list_wp:
    index = unique_list_wp.index(d)
    df_wp_list[index].name = d

for e in df_wp_list:
    name = e.name
    e["WP-JAHRE"] = dict_WPyears[name]
            
for df in df_wp_list:
    df.set_index("WP-JAHRE", drop = False, inplace=True)

# Test
print(df_WP1.index)
print(df_WP19["WP-JAHRE"].index)

C:\Users\Laszlo Kuehl\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Index(['1949-1953;01', '1949-1953;01', '1949-1953;01', '1949-1953;01',
       '1949-1953;01', '1949-1953;01', '1949-1953;01', '1949-1953;01',
       '1949-1953;01', '1949-1953;01',
       ...
       '1949-1953;01', '1949-1953;01', '1949-1953;01', '1949-1953;01',
       '1949-1953;01', '1949-1953;01', '1949-1953;01', '1949-1953;01',
       '1949-1953;01', '1949-1953;01'],
      dtype='object', name='WP-JAHRE', length=474)
Index(['2017-;19', '2017-;19', '2017-;19', '2017-;19', '2017-;19', '2017-;19',
       '2017-;19', '2017-;19', '2017-;19', '2017-;19',
       ...
       '2017-;19', '2017-;19', '2017-;19', '2017-;19', '2017-;19', '2017-;19',
       '2017-;19', '2017-;19', '2017-;19', '2017-;19'],
      dtype='object', name='WP-JAHRE', length=711)


#### Ausführung: 

In [31]:
#Zusammenfügen
df_indexed = pd.concat(df_wp_list)


#Indexing (zusätzlicher numerischer Index als Spalte hinzugefügt --> einfachere Selektion)
num_index = list(range(0,11600))
df_indexed["NUMIND"] = num_index
df_indexed.set_index(["WP-JAHRE"])


#Check
df_indexed.head()                       

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,...,WP12,WP13,WP14,WP15,WP16,WP17,WP18,WP19,WP-JAHRE,NUMIND
WP-JAHRE,,,,,,,,,,,,,,,,,,,,,
1949-1953;01,11000009,1,Union,1876,Bundeskanzler a. D.,Deutschland,Adenauer,Konrad,"[1, 2, 3, 4, 5]","[1, 1, 1, 1, 1]",...,0,0,0,0,0,0,0,0,1949-1953;01,0
1949-1953;01,11000013,1,Andere,1904,"Bergmann, Gewerkschaftsfunktionär",Deutschland,Agatz,Willi,[1],[0],...,0,0,0,0,0,0,0,0,1949-1953;01,1
1949-1953;01,11000015,1,Andere,1879,Kapitän a. D.,Deutschland,Ahrens,Adolf,[1],[0],...,0,0,0,0,0,0,0,0,1949-1953;01,2
1949-1953;01,11000020,1,Union,1890,Geschäftsführer,Deutschland,Albers,Johannes,"[1, 2]","[1, 1]",...,0,0,0,0,0,0,0,0,1949-1953;01,3
1949-1953;01,11000021,0,SPD,1901,Buchhalterin,Deutschland,Albertz,Luise,"[1, 2, 3, 4, 5]","[0, 0, 0, 0, 1]",...,0,0,0,0,0,0,0,0,1949-1953;01,4


### Feature Engneering

#### Erstellen einer Spalte für das Alter der MdBs am Anfang der jeweiligen Wahlperiode.

In [32]:
ANFANG_list = []
for e in df_indexed.index:
    ANFANG_list.append(int(e[:4]))

df_indexed["ANFANG"] = ANFANG_list
df_indexed["ALTER"] = df_indexed["ANFANG"] - df_indexed["GEBURTSDATUM"].apply(int)


# Suche nach vermeidbaren Fehlern 
df_indexed.ALTER.unique() 

array([73, 45, 70, 59, 48, 53, 60, 61, 69, 31, 55, 67, 54, 34, 56, 29, 49,
       58, 44, 36, 42, 66, 39, 43, 51, 38, 52, 37, 65, 64, 47, 50, 57, 46,
       41, 62, 40, 63, 32, 35, 68, 72, 74, 27, 33, 30, 28, 77, 75, 81, 26,
       71, 79, 25, 76, 85, 80, 89, 24, 21, 23, 22, 19, 78], dtype=int64)

#### Erstellen einer Spalte für die Mandatsart der MdBs der jeweiligen Wahlperiode.

In [33]:
# Unschöne Lösung, aber funktioniert
anumindex = df_indexed["NUMIND"]
awpindlst = df_indexed.index.tolist()
awplst = list(df_indexed["WAHLPERIODEN"])
amandlst = list(df_indexed["MANDATSARTEN"])
anewcol = []

for ind in anumindex:
    num = str(int(awpindlst[ind][-2:]))
    if num in awplst[ind]:
        subind = int(awplst[ind].index(num))
        anewcol.append(amandlst[ind][subind])
    

# Test (sollte 11600 Einträge haben)
print(len(anewcol))
# Neue Spalte setzen
df_indexed["MANDATSART"] = anewcol

11600


In [34]:
# Fehler?
df_indexed[df_indexed['MANDATSART'].isnull()]

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME,WAHLPERIODEN,MANDATSARTEN,...,WP15,WP16,WP17,WP18,WP19,WP-JAHRE,NUMIND,ANFANG,ALTER,MANDATSART
WP-JAHRE,,,,,,,,,,,,,,,,,,,,,
1998–2002;14,11002710,1,Union,1955,Dipl.-Ingenieur für Schiffstechnik,Deutschland,Kuhn,Werner,"[13, 14, 15]","[1, None, 0]",...,1,0,0,0,0,1998–2002;14,8023,1998,43,NaN


In [35]:
# Fixing the problem
df_indexed["MANDATSART"].fillna(0, inplace=True) #0, weil kein Direktmandat ersetzt wurde (Siehe Bundestagsarchiv)
df_indexed["MANDATSART"].value_counts()
df_indexed["MANDATSART"].round(0).astype(int)
print("Done")

Done


### Vorbereitung für den Export

#### Letzte Ausgestalungen

In [36]:
# Welche Spalten können gelöscht werden? 
print(df_indexed.columns.values)

# Für die Analyse (Visualisierung/Machine Learning) werden redundante Infos entfernt
df_indexed_final = df_indexed.drop(["GEBURTSDATUM", "GEBURTSLAND", "WAHLPERIODEN","WP-JAHRE", "MANDATSARTEN"], axis=1)


# Sortieren
df_indexed_final = df_indexed_final[
['NUMIND',"ID",'ANFANG', 'VORNAME', 'NACHNAME', 'ALTER', 'GESCHLECHT', 'PARTEI_KURZ',"BERUF","AUSLAND", "MANDATSART", "WP1", "WP2", "WP3", "WP4", "WP5",
       "WP6", "WP7", "WP8", "WP9", "WP10", "WP11", "WP12", "WP13", "WP14",
       'WP15', 'WP16', 'WP17', 'WP18', 'WP19']]

df_indexed_final.head()

['ID' 'GESCHLECHT' 'PARTEI_KURZ' 'GEBURTSDATUM' 'BERUF' 'GEBURTSLAND'
 'NACHNAME' 'VORNAME' 'WAHLPERIODEN' 'MANDATSARTEN' 'AUSLAND' 'WP1' 'WP2'
 'WP3' 'WP4' 'WP5' 'WP6' 'WP7' 'WP8' 'WP9' 'WP10' 'WP11' 'WP12' 'WP13'
 'WP14' 'WP15' 'WP16' 'WP17' 'WP18' 'WP19' 'WP-JAHRE' 'NUMIND' 'ANFANG'
 'ALTER' 'MANDATSART']


,NUMIND,ID,ANFANG,VORNAME,NACHNAME,ALTER,GESCHLECHT,PARTEI_KURZ,BERUF,AUSLAND,...,WP10,WP11,WP12,WP13,WP14,WP15,WP16,WP17,WP18,WP19
WP-JAHRE,,,,,,,,,,,,,,,,,,,,,
1949-1953;01,0,11000009,1949,Konrad,Adenauer,73,1,Union,Bundeskanzler a. D.,0,...,0,0,0,0,0,0,0,0,0,0
1949-1953;01,1,11000013,1949,Willi,Agatz,45,1,Andere,"Bergmann, Gewerkschaftsfunktionär",0,...,0,0,0,0,0,0,0,0,0,0
1949-1953;01,2,11000015,1949,Adolf,Ahrens,70,1,Andere,Kapitän a. D.,0,...,0,0,0,0,0,0,0,0,0,0
1949-1953;01,3,11000020,1949,Johannes,Albers,59,1,Union,Geschäftsführer,0,...,0,0,0,0,0,0,0,0,0,0
1949-1953;01,4,11000021,1949,Luise,Albertz,48,0,SPD,Buchhalterin,0,...,0,0,0,0,0,0,0,0,0,0


### Export des Datensatzes in .csv-Format 

In [37]:
df_indexed_final.to_csv("MdB_Indexed.csv")

#### Abgeschlossen